In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.formula.api as smf
import re

# -------------------- Utility Functions --------------------
def _open_xls(path: str | Path) -> pd.ExcelFile:
    path = Path(path)
    assert path.suffix.lower() in {".xlsx", ".xlsm", ".xls"}, f"Excel file required: {path}"
    return pd.ExcelFile(path)

def _norm(s: str) -> str:
    return "".join(str(s).upper().replace("-", "").replace("_", "").split())

def _pick_sheet(xls: pd.ExcelFile, iso: str) -> str:
    target = _norm(iso)
    for name in xls.sheet_names:
        if _norm(name) == target:
            return name
    for name in xls.sheet_names:
        if target in _norm(name):
            return name
    raise KeyError(f"Sheet not found in {xls} for ISO: {iso}; available sheets: {xls.sheet_names}")

def _find_ts_col(cols) -> str:
    keys = ["timestamp", "time", "datetime", "date", "hour", "interval"]
    for c in cols:
        lc = str(c).lower()
        if any(k in lc for k in keys):
            return c
    return cols[0]  # Fallback: First column

def _to_datetime(series: pd.Series) -> pd.Series:
    dt = pd.to_datetime(series, errors="coerce")
    if dt.isna().all():
        dt = pd.to_datetime(pd.to_numeric(series, errors="coerce"),
                            unit="d", origin="1899-12-30", errors="coerce")
    return dt

def _coerce_numeric(df: pd.DataFrame, except_cols=("ts",)) -> pd.DataFrame:
    num_cols = [c for c in df.columns if c not in except_cols]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
    return df

# -------------------- Table 1: Day-Ahead Prices (Hour x Load Zone, Wide Format) --------------------
def read_table1_prices(path_excel: str | Path, iso: str = "PJM") -> pd.DataFrame:
    xls = _open_xls(path_excel)
    sheet = _pick_sheet(xls, iso)
    df = pd.read_excel(xls, sheet_name=sheet)
    ts_col = _find_ts_col(df.columns)
    df = df.rename(columns={ts_col: "ts"})
    df["ts"] = _to_datetime(df["ts"])
    df = _coerce_numeric(df, except_cols=("ts",)).dropna(subset=["ts"])
    out = df.melt(id_vars="ts", var_name="zone", value_name="lmp_da")
    out["iso"] = iso
    return out[["iso", "zone", "ts", "lmp_da"]].sort_values(["zone", "ts"])

# -------------------- Table 2: Temperature (Hour x Load Zone, Wide Format) --------------------
def read_table2_temperature(path_excel: str | Path, iso: str = "PJM") -> pd.DataFrame:

    xls = _open_xls(path_excel)
    sheet = _pick_sheet(xls, iso)
    df = pd.read_excel(xls, sheet_name=sheet)
    ts_col = _find_ts_col(df.columns)
    df = df.rename(columns={ts_col: "ts"}).copy()
    df["ts"] = pd.to_datetime(df["ts"], errors="coerce")
    if df["ts"].isna().all():
        df["ts"] = pd.to_datetime(pd.to_numeric(df["ts"], errors="coerce"),
                                  unit="d", origin="1899-12-30", errors="coerce")
    df = _coerce_numeric(df, except_cols=("ts",)).dropna(subset=["ts"])
    out = df.melt(id_vars="ts", var_name="zone", value_name="temperature")
    out["iso"] = iso

    return out[["iso", "zone", "ts", "temperature"]].sort_values(["zone", "ts"])

# -------------------- Table 3: ISO Fuel Mix & Gas/Marginal Price (Hour x Fuel, Wide Format) --------------------
def read_table3_fuelmix(path_excel: str | Path, iso: str = "PJM") -> pd.DataFrame:

    strict_last = True
    xls = _open_xls(path_excel)
    sheet = _pick_sheet(xls, iso)
    header = pd.read_excel(xls, sheet_name=sheet, nrows=0)
    cols = list(header.columns)
    ts_col = _find_ts_col(cols)
    def _norm(s: str) -> str:
        return str(s).strip().lower().replace(" ", "").replace("_", "").replace("-", "")
    price_col = None
    if not strict_last:
        for c in cols[::-1]:  # Search from right to left, prefer the last occurrence
            nc = _norm(c)
            if "gasmarginalprice" in nc or "marginalprice" in nc:
                price_col = c
                break
    if price_col is None:
        price_col = cols[-1]  # Force use of last column

    df = pd.read_excel(xls, sheet_name=sheet, usecols=[ts_col, price_col])
    df.columns = ["ts", "marginal_price"]
    df["ts"] = _to_datetime(df["ts"])
    df["marginal_price"] = pd.to_numeric(df["marginal_price"], errors="coerce")
    df = df.dropna(subset=["ts"]).sort_values("ts")
    df["iso"] = iso

    return df[["iso", "ts", "marginal_price"]]

# -------------------- Table 4: Fuel Price (Supplementary) --------------------------
def read_table4_fuel_price(path, iso):

    if path.endswith(".csv"):
        df = pd.read_csv(path)
    else:
        df = pd.read_excel(path)
    df = df.rename(columns={df.columns[0]: "date"})
    df["date"] = pd.to_datetime(df["date"])
    if iso not in df.columns:
        raise ValueError(f"ISO '{iso}' not found in headers, options: {df.columns[1:].tolist()}")
    out = df[["date", iso]].rename(columns={iso: "marginal_price"}).sort_values("date")
    out["iso"] = iso

    return out.reset_index(drop=True)

# -------------------- Table 5: Data Center Cumulative Capacity (Year x Load Zone, Wide Format) --------------------
def read_table5_dc_capacity(path_excel: str | Path, iso: str = "PJM") -> pd.DataFrame:

    xls = _open_xls(path_excel)
    sheet = _pick_sheet(xls, iso)
    df = pd.read_excel(xls, sheet_name=sheet)
    year_col = None
    for c in df.columns:
        if str(c).strip().lower().startswith("year"):
            year_col = c; break
    if year_col is None:
        year_col = df.columns[0]
    df = df.rename(columns={year_col: "year"})
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    df = df[df["year"].ge(2020)].dropna(subset=["year"]).copy()
    zone_cols = [c for c in df.columns if c != "year"]
    df[zone_cols] = df[zone_cols].apply(pd.to_numeric, errors="coerce")
    out = df.melt(id_vars="year", var_name="zone", value_name="dc_cum").dropna(subset=["dc_cum"])
    out["iso"] = iso

    return out[["iso","zone","year","dc_cum"]].sort_values(["zone","year"])

# -------------------- Convenience Wrapper: Read all tables at once --------------------
def load_all_for_iso(paths: dict, iso: str = "PJM") -> dict:
    return { "prices":      read_table1_prices(paths["t1"], iso=iso),
             "temperature": read_table2_temperature(paths["t2"], iso=iso),
             "fuelmix":     read_table3_fuelmix(paths["t3"], iso=iso),
             "dc_capacity": read_table5_dc_capacity(paths["t5"], iso=iso)}

In [ ]:
paths = { "t1": "./tables/price.xlsx",
          "t2": "./tables/temperature_filled.xlsx",
          "t3": "./tables/fuel_marginal.xlsx",
          "t5": "./tables/datacenter_sum.xlsx"}
data_hour = load_all_for_iso(paths, iso="CAISO")

In [ ]:
data = {}
for key in ['prices', 'temperature']:
    df = data_hour[key].copy()
    df['ts'] = pd.to_datetime(df['ts'])
    df['date'] = df['ts'].dt.date
    num_cols = df.select_dtypes(include='number').columns
    daily_df = df.groupby(['iso', 'zone', 'date'])[num_cols].mean().reset_index()
    data[key] = daily_df
df = data_hour['fuelmix'].copy()
df['date'] = pd.to_datetime(df['ts'])
num_cols = df.select_dtypes(include='number').columns
fuelmix_daily = df.groupby(['iso', 'date'])[num_cols].mean().reset_index()
data['fuelmix'] = fuelmix_daily
data["fuelmix"]["marginal_price"] = data["fuelmix"]["marginal_price"].astype(float)
data['dc_capacity'] = data_hour['dc_capacity'].copy()

In [ ]:
temp_df = data["temperature"].copy()
BASE_HEAT = 65.0
BASE_COOL = 65.0
HDD_SCALE = 100
CDD_SCALE = 100
T = pd.to_numeric(temp_df["temperature"], errors="coerce")
hdd_raw = (BASE_HEAT - T).clip(lower=0)
cdd_raw = (T - BASE_COOL).clip(lower=0)
temp_df["HDD"] = (hdd_raw / HDD_SCALE).astype(float)
temp_df["CDD"] = (cdd_raw / CDD_SCALE).astype(float)
temp_df["HDD2"] = temp_df["HDD"] ** 2
temp_df["CDD2"] = temp_df["CDD"] ** 2
data["temperature"] = temp_df

In [ ]:
price_df = data["prices"].copy()
price_df = price_df.sort_values(["zone","date"])
first_price = price_df.groupby("zone")["lmp_da"].transform("first")
price_df["price_diff"] = price_df["lmp_da"]
data["prices"] = price_df

In [ ]:
# %% 1) Select ISO, perform basic cleaning and merging
iso_tag = "CAISO"
prices = data["prices"].query("iso == @iso_tag").copy()
temps  = data["temperature"].query("iso == @iso_tag")[["iso","zone","date","temperature","HDD","CDD","HDD2","CDD2" ]].copy()
mix    = data["fuelmix"].query("iso == @iso_tag")[["iso","date","marginal_price"]].copy()
dc     = data["dc_capacity"].query("iso == @iso_tag").copy()
# Time decomposition & month x hour fixed effect index (0~287)
prices["date"] = pd.to_datetime(prices["date"], errors="coerce")
temps["date"] = pd.to_datetime(temps["date"], errors="coerce")
mix["date"] = pd.to_datetime(mix["date"], errors="coerce")
prices["year"] = prices["date"].dt.year
prices["month"] = prices["date"].dt.month
# Merge temperature and fuel marginal cost controls (X_{z,t})
df = ( prices.merge(temps, on=["iso","zone","date"], how="left").merge(mix,on=["iso","date"],how="left"))

In [ ]:
# %% 2) Construct Data Center variables (Daily linear interpolation within year, reaching cumulative total at year-end)
start_date = pd.Timestamp("2020-01-01")
end_date   = pd.Timestamp("2025-07-30")
df = df[(df["date"] >= start_date) & (df["date"] <= end_date)].copy()
df["year"] = df["date"].dt.year
keys_local = df[["iso", "zone", "year"]].drop_duplicates()
keys_iso   = df[["iso", "year"]].drop_duplicates()
dc_y = dc[(dc["year"] >= 2020) & (dc["year"] <= 2025)].copy()
dc_local_raw = dc_y[["iso", "zone", "year", "dc_cum"]].copy()
dc_2020_25 = (keys_local.merge(dc_local_raw, on=["iso","zone","year"], how="left").sort_values(["iso","zone","year"]))

# —— 1) Prepare annual "Target (Year-end Cumulative)" and "Baseline (Previous Year-end Cumulative)" —— #
# Local: iso + zone dimension
dc_local_year = (dc_2020_25.rename(columns={"dc_cum": "dc_local_target"})[["iso", "zone", "year", "dc_local_target"]].sort_values(["iso","zone","year"]))
dc_local_year["dc_local_prev"] = (dc_local_year.groupby(["iso","zone"])["dc_local_target"].shift(1).fillna(0.0))
dc_local_year = dc_local_year.fillna(0.0)
# ISO Total: iso dimension
dc_iso_year = (dc_2020_25.groupby(["iso","year"], as_index=False)["dc_cum"].sum().rename(columns={"dc_cum":"dc_iso_target"}).sort_values(["iso","year"]))
dc_iso_year["dc_iso_prev"] = (dc_iso_year.groupby("iso")["dc_iso_target"].shift(1).fillna(0.0))
dc_iso_year = dc_iso_year.fillna(0.0)

# —— 2) Merge "Target/Baseline" into hourly dataframe —— #
df = (df.merge(dc_local_year, on=["iso","zone","year"], how="left").merge(dc_iso_year,on=["iso","year"],how="left"))
for col in ["dc_local_target","dc_local_prev","dc_iso_target","dc_iso_prev"]:
    df[col] = df[col].fillna(0.0)
# If target is missing for a year, keep same as previous (no growth)
for cur_col, prev_col in [("dc_local_target","dc_local_prev"),("dc_iso_target","dc_iso_prev")]:
    df[cur_col] = df[cur_col].fillna(df[prev_col])
    df[prev_col] = df[prev_col].fillna(0.0)

# —— 3) Calculate "Intra-year Progress Coefficient" (Jan 1 = 0, Dec 31 = 1; handles leap years) —— #
year_start = pd.to_datetime(df["year"].astype(str) + "-01-01")
year_end   = pd.to_datetime(df["year"].astype(str) + "-12-31")
# Use "day" as unit; normalize hourly data to date before calculating day index
d0 = df["date"].dt.normalize()
elapsed_days = (d0 - year_start).dt.days.astype("int64")            # 0,1,...,365/366-1
total_days   = (year_end - year_start).dt.days.astype("int64")      # 365 or 366
frac = (elapsed_days / total_days).clip(lower=0.0, upper=1.0)       # [0,1]

# —— 4) Linear interpolation within the year —— #
df["dc_local"]    = df["dc_local_prev"] + (df["dc_local_target"] - df["dc_local_prev"]) * frac
df["dc_iso_total"] = df["dc_iso_prev"]  + (df["dc_iso_target"]  - df["dc_iso_prev"])  * frac

# —— 5) External volume (Other zones in same ISO) & Unit Conversion —— #
df["dc_external"] = (df["dc_iso_total"] - df["dc_local"]).clip(lower=0.0)
# Keep float and apply 1/1000 conversion
scale_cols1 = ["dc_local"]
df[scale_cols1] = df[scale_cols1].astype(float) / 1000.0
scale_cols2 = ["dc_iso_total", "dc_external"]
df[scale_cols2] = df[scale_cols2].astype(float) / 1000.0

In [ ]:
# %% 3) Select columns for regression and perform minimal cleaning
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["dow"] = df["date"].dt.dayofweek
need_cols = ["price_diff", "dc_local","dc_external","dc_iso_total", "temperature",'HDD','CDD','HDD2','CDD2', "marginal_price","zone","month","year","day",'dow','date' ]
reg = df[need_cols].dropna().copy()
print(f"Sample size for regression: {len(reg):,} rows, Zones: {reg['zone'].nunique()}.")

In [ ]:
def winsorize_by_group(d, cols, ql=0.005, qh=0.995, by="zone"):

    g = d.groupby(by)
    lo = g[cols].transform(lambda s: s.quantile(ql))
    hi = g[cols].transform(lambda s: s.quantile(qh))
    in_range = (d[cols] >= lo) & (d[cols] <= hi)
    keep = in_range.all(axis=1)

    return d.loc[keep].copy()

In [ ]:
THRESH = 0.000
# —— 1) Zone Filtering: Exclude zones where dc_local < 0.01 throughout the period ——
reg = reg.copy()
reg["dc_local"] = pd.to_numeric(reg["dc_local"], errors="coerce").fillna(0.0)
zone_max = reg.groupby("zone")["dc_local"].apply(lambda s: s.abs().max())
keep_zones = sorted(zone_max[zone_max > THRESH].index.tolist())
drop_zones = sorted(zone_max[zone_max <= THRESH].index.tolist())
#reg1 = reg.loc[reg["zone"].isin(keep_zones)].copy()
reg1 = winsorize_by_group(reg.loc[reg["zone"].isin(keep_zones)].copy(), ["price_diff"], 0.05, 0.95, "zone")
reg2 = winsorize_by_group(reg1, ["marginal_price"], 0.01, 0.99, "zone")

zone_mean = pd.to_numeric(reg["price_diff"], errors="coerce").groupby(reg["zone"]).mean()
overall_mean_plus = float(zone_mean.mean())
print(f"Overall mean after zone aggregation: {overall_mean_plus:.6f}")

In [ ]:
# ===== Config: Change to your output directory and desired data table =====
OUT_DIR = Path("./fitting_result")       # Your output folder
SRC = reg                               # Use winsorized result; change to 'reg' if raw data is preferred
# ===== Calculate Mean per Zone =====
zone_mean_df = (
    pd.to_numeric(SRC["price_diff"], errors="coerce")
      .groupby(SRC["zone"]).mean()
      .rename("price_diff_mean")
      .reset_index()
      .sort_values("zone")
)
zone_mean_df["price_diff_mean"] = zone_mean_df["price_diff_mean"].round(6)
# Try to get ISO name from data; default to "ISO"
iso_name = iso_tag
# ===== Write to Excel: Separate sheets by ISO in the same file =====
OUT_DIR.mkdir(parents=True, exist_ok=True)
out_file = OUT_DIR / "zone_price_diff_means.xlsx"   # You can change the filename
# ===== Write to Excel: Append/Replace sheet if exists; create new if not =====
if out_file.exists():
    with pd.ExcelWriter(out_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        zone_mean_df.to_excel(writer, sheet_name=str(iso_name), index=False)
else:
    with pd.ExcelWriter(out_file, engine="openpyxl", mode="w") as writer:
        zone_mean_df.to_excel(writer, sheet_name=str(iso_name), index=False)

print(f"Written to {out_file} -> Sheet: {iso_name}")

In [ ]:
from linearmodels.panel import PanelOLS
#
reg2["date"] = pd.to_datetime(reg1["date"])   # Skip if already datetime
panel = reg2.set_index(["zone", "date"]).sort_index()
panel = panel.assign(zone=lambda d: d.index.get_level_values("zone"))
res_dk = PanelOLS.from_formula("price_diff ~ 1 + HDD2 + CDD2 + C(zone):marginal_price + dc_local + C(year) + C(month):C(dow) + EntityEffects",
    data=panel).fit(cov_type="kernel", kernel="bartlett", bandwidth=14)
print(res_dk.summary)

In [ ]:
import os

def result_to_df(res):
    df = pd.DataFrame({
        "coef": res.params,
        "std_err": res.std_errors,
        "t_stat": res.tstats,
        "pval": res.pvalues
    })
    ci = res.conf_int(level=0.95)
    ci.columns = ["ci_lower", "ci_upper"]
    return pd.concat([df, ci], axis=1)


# Coefficients
res_dk_df = result_to_df(res_dk)
file_path = "./fitting_result/res_dk_results.xlsx"
if not os.path.exists(file_path):
    with pd.ExcelWriter(file_path, mode="w", engine="openpyxl") as writer:
        res_dk_df.to_excel(writer, sheet_name=iso_tag)
else:
    with pd.ExcelWriter(file_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        res_dk_df.to_excel(writer, sheet_name=iso_tag)